In [1]:
import torch
from transformers import pipeline
from tqdm import tqdm
import swifter

import pandas as pd






#pip install torch transformers tqdm swifter pandas IProgress 
# python -m spacy download en_core_web_sm

/home/ucloud/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


This script runs Burnhams DEBATE model for blame detection on the translated text and maps that back to the original Danish utterance.

In [4]:
# Here we define a variable that will be passed to our classifier. This will check if a GPU is available, and use the CPU if one is not available.
device = "cuda" if torch.cuda.is_available() else "cpu" # if you want to use the GPU on a macbook change 'cuda' to 'mps' and make sure you have the 'accelerate' library installed.
# This line prints the device that will be used. Make sure it prints 'cuda' or 'mps' if you are trying to use a GPU.

print(f"Device: {device}")

Device: cuda


In [6]:
#pipe = pipeline("zero-shot-classification", model="mlburnham/Political_DEBATE_base_v1.0", device = device, batch_size = 32) # To use the base model
pipe = pipeline("zero-shot-classification", model='mlburnham/Political_DEBATE_large_v1.0', device = device, batch_size = 128) # To use the large model

Device set to use cuda


In [7]:
test_doc = ' it is the fault of the immigrants I think but maybe it is not entirely their fault hello'
hypothesis_template = "Based on this text, the author's attitude towards others is best described as {}."
test_labels = ["blame", "praise", "neutral"]

def evaluate_premise(doc, labels, hypothesis_template):
    
    result = pipe(doc, labels, hypothesis_template = hypothesis_template, multi_label = True)
    result.pop('sequence')

    return result

result = evaluate_premise(test_doc, test_labels, hypothesis_template)

In [8]:
result

{'labels': ['blame', 'neutral', 'praise'],
 'scores': [0.9992548227310181, 0.9989248514175415, 1.2478437838581158e-07]}

In [5]:
# Load

import pandas as pd

df_trans = pd.read_csv("/work/MarkusLundsfrydJensen#1865/annotation_data_translated_version_03_10.csv")

df_trans.pop('Unnamed: 0')




df_trans.head()

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country,da_segmented_text,translated_text
0,1997-10-07,Dagsorden,1,Gert Petersen,NaN,NaN,True,191,Mødet er åbnet. I henhold til grundloven er Fo...,DK-Folketing,DNK,"['Mødet er åbnet.', 'I henhold til grundloven ...","['The meeting is open.', 'Under the Constituti..."
1,1997-10-07,Dagsorden,2,Formanden,NaN,NaN,True,182,"Jeg vil gerne takke Tinget for den tillid, man...",DK-Folketing,DNK,"['Jeg vil gerne takke Tinget for den tillid, m...",['I would like to thank Things for the confide...
2,1997-10-07,Statsministerens redegørelse i henhold til gru...,3,Poul Nyrup Rasmussen,S,379.0,False,18662,For 25 år siden sagde et flertal i befolkninge...,DK-Folketing,DNK,['For 25 år siden sagde et flertal i befolknin...,"['Twenty-five years ago, a majority of the peo..."
3,1997-10-09,1) Indstilling fra Udvalget til Valgs Prøvelse.,2,Formanden,NaN,NaN,True,47,Fra Udvalget til Valgs Prøvelse har jeg modtag...,DK-Folketing,DNK,"['Fra Udvalget til Valgs', 'Prøvelse', 'har je...",['From the Committee to the Committee of the R...
4,1997-10-09,2) Forhandling om redegørelse nr. R 1.,3,Torben Lund,S,379.0,False,2865,Vi står over for en meget afgørende folketings...,DK-Folketing,DNK,['Vi står over for en meget afgørende folketin...,['We are faced with a very crucial parliamenta...


In [11]:
len(df_trans)

36314

In [ ]:
#test_doc = ' it is the fault of the immigrants I think but maybe it is not entirely their fault hello'


#result = evaluate_premise(test_doc, test_labels, hypothesis_template)

In [2]:
import ast


In [ ]:
#tried to make it faster


# Pick device automatically
device = 0 if torch.cuda.is_available() else -1
print(f"Device: {'cuda' if device == 0 else 'cpu'}")

# Use base model for speed (large is much slower)
pipe = pipeline("zero-shot-classification", model='mlburnham/Political_DEBATE_large_v1.0', device = device, batch_size = 512)

'''def split_paragraph(paragraph: str):
    """Split paragraph into sentences."""
    doc = nlp(paragraph)
    return [sent.text.strip() for sent in doc.sents]'''

# Template + labels
hypothesis_template = "Based on this text, the author's attitude towards others is best described as {}."
labels = ["blame", "praise", "neutral"]

def evaluate_premise(doc):
    return pipe(doc, labels, hypothesis_template=hypothesis_template, multi_label=True)

def blame_in_batch(sentences, batch_size=512):
    results = []
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i+batch_size]
        output = evaluate_premise(batch)
        if isinstance(output, list):
            results.extend(output)
        else:
            results.append(output)
    return results

def blame_in_paragraph(para):
    #sentences = split_paragraph(para)
    sentences = ast.literal_eval(para)
    return blame_in_batch(sentences)

'''# ✅ Apply in parallel with 

def detect_blame_in_dataframe(df, text_col="translated_text", batch_size=512):
    tqdm.pandas(desc="Blame Detection (GPU)")
    df["blame_in_text"] = df[text_col].swifter.apply(blame_in_paragraph)
    return df'''

def detect_blame_in_dataframe(df, text_col="translated_text", output_column = "blame_in_text"):
    results = []
    for para in tqdm(df[text_col], desc="Blame Detection (GPU)"):
        results.append(blame_in_paragraph(para))
    df[output_column] = results
    return df
# Example usage
# data_test = detect_blame_in_dataframe(data_test)


Device: cuda


Device set to use cuda:0


In [6]:
data_test = df_trans.loc[:5]

In [23]:
#data_test = detect_blame_in_dataframe(data_test)
df_trans = detect_blame_in_dataframe(df_trans)



Blame Detection (GPU): 100%|██████████| 36314/36314 [1:19:48<00:00,  7.58it/s]  


In [24]:
#clean data columns
export_data = df_trans
export_data['da_segmented_text'] = export_data['da_segmented_text'].apply(ast.literal_eval)
export_data['translated_text'] = export_data['translated_text'].apply(ast.literal_eval)

In [25]:
export_data.to_csv("/work/MarkusLundsfrydJensen#1865/cleaned_annotation_data_final.csv")

In [26]:
#test if the cleaning worked

type(export_data.loc[0]['da_segmented_text'])


list

In [27]:
type(export_data.loc[0]['translated_text'])

list

In [55]:
from transformers import MarianMTModel, MarianTokenizer

model_name = 'Helsinki-NLP/opus-mt-da-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

def translate(text):
    translated = model.generate(**tokenizer(text, return_tensors="pt", padding=True))
    return tokenizer.decode(translated[0], skip_special_tokens=True)

print(translate("Hej, hvordan har du det?"))


ModuleNotFoundError: No module named 'transformers'

In [14]:
import spacy 

nlp = spacy.load("da_core_news_sm")

def split_paragraph(paragraph: str):
    doc = nlp(paragraph)
    sentences = [sent.text.strip() for sent in doc.sents]
    return sentences

In [15]:
def translate_paragraph(para):

    translated_sentences = ""

    sentences = split_paragraph(para)

    for sent in sentences:
        #translate sentence
        english_sentence = translate(sent)

        translated_sentences += english_sentence + " "

    return translated_sentences
        


In [7]:
paragraph = "Hej med dig, hvordan går det? Du kan tro, jeg har oplevet meget de seneste par dage. Jeg har super travlt, men ellers går det fint. Jeg var nede og handle den anden dag. Det var virkelig hårdt at bære så mange ting op i lejligheden igen. Hej med dig, hvordan går det? Du kan tro, jeg har oplevet meget de seneste par dage. Jeg har super travlt, men ellers går det fint. Jeg var nede og handle den anden dag. Det var virkelig hårdt at bære så mange ting op i lejligheden igen. Hej med dig, hvordan går det? Du kan tro, jeg har oplevet meget de seneste par dage. Jeg har super travlt, men ellers går det fint. Jeg var nede og handle den anden dag. Det var virkelig hårdt at bære så mange ting op i lejligheden igen. Hej med dig, hvordan går det? Du kan tro, jeg har oplevet meget de seneste par dage. Jeg har super travlt, men ellers går det fint. Jeg var nede og handle den anden dag. Det var virkelig hårdt at bære så mange ting op i lejligheden igen4 Hej med dig, hvordan går det? Du kan tro, jeg har oplevet meget de seneste par dage. Jeg har super travlt, men ellers går det fint. Jeg var nede og handle den anden dag. Det var virkelig hårdt at bære så mange ting op i lejligheden igen. 7."

output = translate_paragraph(paragraph)

output

"Hey, you. How's it going? You bet I've been through a lot these past few days. I'm super busy, but otherwise I'll be fine. I was down shopping the other day. It was really hard to carry so many things up the apartment again. Hey, you. How you doing? You bet I've been through a lot these past few days. I'm super busy, but otherwise I'll be fine. I was down shopping the other day. It was really hard to carry so many things up the apartment again. Hey, you. How you doing? You bet I've been through a lot these past few days. I'm super busy, but otherwise I'll be fine. I was down shopping the other day. It was really hard to carry so many things up the apartment again. Hey, you. How you doing? You bet I've been through a lot these past few days. I'm super busy, but otherwise I'll be fine. I was down shopping the other day. It was really hard to carry so many things up in the apartment again4 Hey, you. How's it going? You bet I've been through a lot these past few days. I'm super busy, but 

In [23]:
#df['translated_text'] = df['text'].apply(translate_paragraph)

import swifter

df['translated_text'] = df['text'].swifter.apply(translate_paragraph)

KeyboardInterrupt: 

In [ ]:
import torch
from transformers import MarianMTModel, MarianTokenizer
import spacy
import swifter

# Load models
model_name = 'Helsinki-NLP/opus-mt-da-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
model.eval()
if torch.cuda.is_available():
    model.to('cuda')

# SpaCy for sentence splitting (optional)
nlp = spacy.load("da_core_news_sm")

def split_paragraph(paragraph: str):
    doc = nlp.pipe([paragraph], batch_size=1, n_process=1)
    return [sent.text.strip() for sent in list(doc)[0].sents]

def translate_batch(sentences, batch_size=16):
    translations = []
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i+batch_size]
        tokens = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
        if torch.cuda.is_available():
            tokens = {k: v.to('cuda') for k, v in tokens.items()}
        translated = model.generate(**tokens)
        decoded = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
        translations.extend(decoded)
    return translations

def translate_paragraph(para):
    sentences = split_paragraph(para)
    translated_sentences = translate_batch(sentences)
    return " ".join(translated_sentences)

# Apply to dataframe in parallel
#df['translated_text'] = df['text'].swifter.apply(translate_paragraph)


In [9]:
df['translated_text'] = df['text'].swifter.apply(translate_paragraph)

KeyboardInterrupt: 

In [3]:
!python -m spacy download da_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 21.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('da_core_news_sm')


In [1]:
print('hello0')

hello0


In [12]:
import torch
from tqdm import tqdm
from transformers import MarianMTModel, MarianTokenizer
import spacy
from torch.cuda.amp import autocast
import IProgress
import ipywidgets

# Load model on GPU
model_name = "Helsinki-NLP/opus-mt-da-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to("cuda").eval()

# Load spaCy Danish model
nlp = spacy.load("da_core_news_sm", disable=["tagger", "ner", "lemmatizer"])

def split_paragraph(paragraph: str):
    """Split paragraph into sentences."""
    doc = nlp(paragraph)
    return [sent.text.strip() for sent in doc.sents]

def translate_batch(sentences, batch_size=128):
    """Translate batch of sentences on GPU with mixed precision."""
    translations = []
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i+batch_size]
        tokens = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to("cuda")
        with torch.no_grad(), autocast():
            translated = model.generate(**tokens, max_length=128)
        decoded = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
        translations.extend(decoded)
    return translations

def translate_paragraph(para):
    """Translate a paragraph sentence by sentence."""
    sentences = split_paragraph(para)
    translated_sentences = translate_batch(sentences)
    return sentences, translated_sentences

def translate_dataframe_gpu(df, text_col="text", batch_size=128):
    """Translate DataFrame sequentially with sentence splitting."""
    texts = df[text_col].tolist()
    translated_texts = []
    untranslated_text = []

    for p in tqdm(texts, desc="GPU Translation"):
        untranslated_sentences, translated_sentences = translate_paragraph(p)

        translated_texts.append(translated_sentences)
        untranslated_text.append(untranslated_sentences)

    df['da_segmented_text'] = untranslated_text
    df["translated_text"] = translated_texts
    return df


In [8]:
df_test = df_trans.loc[:5]

In [19]:
df_trans = translate_dataframe_gpu(df_trans, text_col="text", batch_size=128)

GPU Translation:   0%|          | 0/36314 [00:00<?, ?it/s]/tmp/ipykernel_30142/3330344898.py:28: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast():
GPU Translation: 100%|██████████| 36314/36314 [3:09:40<00:00,  3.19it/s]  


In [20]:
df_trans.to_csv("/work/MarkusLundsfrydJensen#1865/annotation_data_translated_version_03_10.csv")

In [16]:
df_test

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country,da_segmented_text,translated_text
0,1997-10-07,Dagsorden,1,Gert Petersen,NaN,NaN,True,191,Mødet er åbnet. I henhold til grundloven er Fo...,DK-Folketing,DNK,"[Mødet er åbnet., I henhold til grundloven er ...","[The meeting is open., Under the Constitution,..."
1,1997-10-07,Dagsorden,2,Formanden,NaN,NaN,True,182,"Jeg vil gerne takke Tinget for den tillid, man...",DK-Folketing,DNK,"[Jeg vil gerne takke Tinget for den tillid, ma...",[I would like to thank Things for the confiden...
2,1997-10-07,Statsministerens redegørelse i henhold til gru...,3,Poul Nyrup Rasmussen,S,379.0,False,18662,For 25 år siden sagde et flertal i befolkninge...,DK-Folketing,DNK,[For 25 år siden sagde et flertal i befolkning...,"[Twenty-five years ago, a majority of the peop..."
3,1997-10-09,1) Indstilling fra Udvalget til Valgs Prøvelse.,2,Formanden,NaN,NaN,True,47,Fra Udvalget til Valgs Prøvelse har jeg modtag...,DK-Folketing,DNK,"[Fra Udvalget til Valgs, Prøvelse, har jeg mod...",[From the Committee to the Committee of the Re...
4,1997-10-09,2) Forhandling om redegørelse nr. R 1.,3,Torben Lund,S,379.0,False,2865,Vi står over for en meget afgørende folketings...,DK-Folketing,DNK,[Vi står over for en meget afgørende folketing...,[We are faced with a very crucial parliamentar...
5,1997-10-09,2) Forhandling om redegørelse nr. R 1.,4,Formanden,NaN,NaN,True,211,Der er nu bedt om 17 korte bemærkninger. ...,DK-Folketing,DNK,"[Der er nu bedt om 17 korte bemærkninger., Og ...","[17 brief comments have now been asked., And n..."


In [28]:
for i in range(len(export_data)):
    temp_data = export_data.loc[i]
    if len(temp_data['da_segmented_text']) != len(temp_data['translated_text']):
        print(temp_data)

In [11]:
df_trans.to_csv("/work/MarkusLundsfrydJensen#1865/annotation_data_translated.csv")

In [ ]:
#Take blame text, split danish text into sentences in the same way as the translation

In [8]:
import pandas as pd
import spacy

In [ ]:
#load data
#data = pd.read_csv("/work/MarkusLundsfrydJensen#1865/cleaned_annotation_data_translated_with_blame_heuristics.csv")

In [2]:
#Do second run of blame detection

import pandas as pd

df_final = pd.read_csv("/work/MarkusLundsfrydJensen#1865/annotation_data_second_template_appended.csv")

df_final.pop('Unnamed: 0')


df_final.head()



,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country,da_segmented_text,translated_text,blame_in_text,second_template_blame_in_text
0,1997-10-07,Dagsorden,1,Gert Petersen,NaN,NaN,True,191,Mødet er åbnet. I henhold til grundloven er Fo...,DK-Folketing,DNK,"['Mødet er åbnet.', 'I henhold til grundloven ...","['The meeting is open.', 'Under the Constituti...","[{'sequence': 'The meeting is open.', 'labels'...","[{'sequence': 'The meeting is open.', 'labels'..."
1,1997-10-07,Dagsorden,2,Formanden,NaN,NaN,True,182,"Jeg vil gerne takke Tinget for den tillid, man...",DK-Folketing,DNK,"['Jeg vil gerne takke Tinget for den tillid, m...",['I would like to thank Things for the confide...,[{'sequence': 'I would like to thank Things fo...,[{'sequence': 'I would like to thank Things fo...
2,1997-10-07,Statsministerens redegørelse i henhold til gru...,3,Poul Nyrup Rasmussen,S,379.0,False,18662,For 25 år siden sagde et flertal i befolkninge...,DK-Folketing,DNK,['For 25 år siden sagde et flertal i befolknin...,"['Twenty-five years ago, a majority of the peo...","[{'sequence': 'Twenty-five years ago, a majori...","[{'sequence': 'Twenty-five years ago, a majori..."
3,1997-10-09,1) Indstilling fra Udvalget til Valgs Prøvelse.,2,Formanden,NaN,NaN,True,47,Fra Udvalget til Valgs Prøvelse har jeg modtag...,DK-Folketing,DNK,"['Fra Udvalget til Valgs', 'Prøvelse', 'har je...",['From the Committee to the Committee of the R...,[{'sequence': 'From the Committee to the Commi...,[{'sequence': 'From the Committee to the Commi...
4,1997-10-09,2) Forhandling om redegørelse nr. R 1.,3,Torben Lund,S,379.0,False,2865,Vi står over for en meget afgørende folketings...,DK-Folketing,DNK,['Vi står over for en meget afgørende folketin...,['We are faced with a very crucial parliamenta...,[{'sequence': 'We are faced with a very crucia...,[{'sequence': 'We are faced with a very crucia...


In [ ]:
Set 1 – Direct attitude framing (closest to original)
"Overall, the author’s stance toward others in this passage is {}."

🔹 Set 2 – Attribution and evaluation framing
"The overall feeling that the author communicates toward others in this text, is best decribed as {}."

🔹 Set 3 – Causal / responsibility framing (captures blame nuance more strongly)
"According to this passage, the author’s reaction to others’ conduct can be described as {}."
→ Good causal framing with explicit reaction attribution.

Set 4 – Moral / emotional framing (more affective dimension)

"From the way others are described, the author's expression towards them is best described as {}."

In [10]:
#tried to make it faster

import ast

# Pick device automatically
device = 0 if torch.cuda.is_available() else -1
print(f"Device: {'cuda' if device == 0 else 'cpu'}")

# Use base model for speed (large is much slower)
pipe = pipeline("zero-shot-classification", model='mlburnham/Political_DEBATE_large_v1.0', device = device, batch_size = 256)


# Template + labels
hypothesis_template = "From the way others are described, the author's expression towards them is best described as {}."
labels = ["blame", "praise", "neutral"]

def evaluate_premise(doc):
    return pipe(doc, labels, hypothesis_template=hypothesis_template, multi_label=True)

def blame_in_batch(sentences, batch_size=256):
    results = []
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i+batch_size]
        output = evaluate_premise(batch)
        if isinstance(output, list):
            results.extend(output)
        else:
            results.append(output)
    return results

def blame_in_paragraph(para):
    sentences = ast.literal_eval(para)
    return blame_in_batch(sentences)


def detect_blame_in_dataframe(df, text_col="translated_text", output_column = "fifth_template_blame_in_text"):
    results = []
    for para in tqdm(df[text_col], desc="Blame Detection (GPU)"):
        results.append(blame_in_paragraph(para))
    df[output_column] = results
    return df
# Example usage
# data_test = detect_blame_in_dataframe(data_test)


Device: cuda


Device set to use cuda:0


In [ ]:
#test_data = df_final.loc[:5]

In [11]:
df_final = detect_blame_in_dataframe(df_final)

Blame Detection (GPU): 100%|██████████| 36314/36314 [1:20:54<00:00,  7.48it/s]  


In [12]:
df_final.to_csv("/work/MarkusLundsfrydJensen#1865/annotation_data_fifth_template_appended.csv")

In [1]:
df_final

NameError: name 'df_final' is not defined